In [1]:
import jieba

In [2]:
def readLines(path):
    result = []
    file = open(path,'r', encoding='UTF-8')
    lines = file.readlines()
    
    temp = 0
    for line in lines:
        temp += 1
        if(temp<len(lines)):
            result.append(line[:-1])
    return result

In [3]:
# 分词，去除停用词
def sen2word(sentence):
    segList = jieba.cut(sentence)
    preSegResult = []
    for seg in segList:
        preSegResult.append(seg)

    stopwords = readLines('./dict/哈工大停用词表扩展.txt')
    
    segResult = []
    for seg in preSegResult:
        if seg in stopwords:
            continue
        else:
            segResult.append(seg)
      
    return segResult

In [4]:
sen2word('你好吗，我很好，只是有点小感冒')
# Input: '你好吗，我很好，只是有点小感冒'
# preSegResult: ['你好', '吗', '，', '我', '很', '好', '，', '只是', '有点', '小', '感冒']
# segResult:['你好', '很', '好', '有点', '小', '感冒']

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\pc\AppData\Local\Temp\jieba.cache
Loading model cost 1.015 seconds.
Prefix dict has been built succesfully.


['你好', '很', '好', '有点', '小', '感冒']

In [5]:
# list转dict
def list_to_dict(word_list):
    data = {}
    for x in range(0, len(word_list)):
        data[word_list[x]] = x
    return data

In [9]:
_# 将词语分类并记录其位置
def classify_words(word_dict):
    # 情感词字典
    sen_list = readLines('./dict/BosonNLP_sentiment_score.txt')
    sen_dict = {}
    for s in sen_list:
        sen_dict[s.split(' ')[0]] = s.split(' ')[1]
    # 否定词字典
    not_word_list = readLines('./dict/否定词典/否定.txt')
    # 程度副词字典
    degree_list = readLines('./dict/知网Hownet情感词典/程度级别词语（中文）.txt')
    degree_dic = {}
    for d in degree_list:
        degree_dic[d.split(',')[0]] = d.split(',')[1]
    
    # 分类
    sen_word = dict()
    not_word = dict()
    degree_word = dict()
    
    for word in word_dict.keys():
        if word in sen_dict.keys() and word not in not_word_list and word not in degree_dic.keys():
            # 找出分词结果中在情感字典中的词
            sen_word[word_dict[word]] = sen_dict[word]
        elif word in not_word_list and word not in degree_dic.keys():
            # 分词结果中在否定词列表中的词
            not_word[word_dict[word]] = -1
        elif word in degree_dic.keys():
            # 分词结果中在程度副词中的词
            degree_word[word_dict[word]] = degree_dic[word]
    
    return sen_word, not_word, degree_word

In [10]:
classify_words({'你好':0, '很':0, '好':0, '有点':0, '小':0, '感冒':0})

({0: '-2.7582653803'}, {}, {0: '1.25'})

In [15]:
def get_init_weight(sen_word, not_word, degree_word):
    # 权重初始化为1
    W = 1
    # 将情感字典的key转为list
    sen_word_index_list = list(sen_word.keys())
    if len(sen_word_index_list) == 0:
        return W
    # 获取第一个情感词的下标，遍历从0到此位置之间的所有词，找出程度词和否定词
    for i in range(0, sen_word_index_list[0]):
        if i in not_word.keys():
            W *= -1
        elif i in degree_word.keys():
            # 更新权重，如果有程度副词，分值乘以程度副词的程度分值
            W *= float(degree_word[i])
    return W

In [16]:
def socre_sentiment(sen_word, not_word, degree_word, seg_result):
    """计算得分"""
    # 权重初始化为1
    W = 1
    score = 0
    # 情感词下标初始化
    sentiment_index = -1
    # 情感词的位置下标集合
    sentiment_index_list = list(sen_word.keys())
    # 遍历分词结果(遍历分词结果是为了定位两个情感词之间的程度副词和否定词)
    for i in range(0, len(seg_result)):
        # 如果是情感词（根据下标是否在情感词分类结果中判断）
        if i in sen_word.keys():
            # 权重*情感词得分
            score += W * float(sen_word[i])
            # 情感词下标加1，获取下一个情感词的位置
            sentiment_index += 1
            if sentiment_index < len(sentiment_index_list) - 1:
                # 判断当前的情感词与下一个情感词之间是否有程度副词或否定词
                for j in range(sentiment_index_list[sentiment_index], sentiment_index_list[sentiment_index + 1]):
                    # 更新权重，如果有否定词，取反
                    if j in not_word.keys():
                        W *= -1
                    elif j in degree_word.keys():
                        # 更新权重，如果有程度副词，分值乘以程度副词的程度分值
                        W *= float(degree_word[j])
        # 定位到下一个情感词
        if sentiment_index < len(sentiment_index_list) - 1:
            i = sentiment_index_list[sentiment_index + 1]
    return score

In [17]:
def setiment_score(sententce):
    seg_list = sen2word(sententce)
    sen_word, not_word, degree_word = classify_words(list_to_dict(seg_list))
    score = socre_sentiment(sen_word, not_word, degree_word, seg_list)
    return score

In [18]:
setiment_score('你好吗，我很好，只是有点小感冒')

-1.1884379423780018